In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm

In [ ]:
lens_width = 15.0

class Optics(tlm.Module):
    def __init__(self):
        super().__init__()

        self.surface1 = tlm.BezierSpline(width=lens_width, init=([3.0], [0.2*lens_width, 1.2*lens_width], [4.8]))

        # TODO enforce CX > X, i.e. control point within knots

        self.optics = nn.Sequential(
            tlm.ParallelBeamUniform(width=0.99*lens_width, num_rays=10),
            tlm.Gap(15.),
            
            tlm.RefractiveSurface(self.surface1, (1.0, 1.49)),
            tlm.Gap(15.),
            tlm.RefractiveSurface(self.surface1, (1.49, 1.0), scale=-1),
            
            tlm.Gap(12.0),
            tlm.FocalPointLoss(),
        )

    def forward(self, inputs):
        return self.optics(inputs)

optics = Optics()


tlm.render_plt(optics)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), 5e-2),
    inputs = init,
    num_iter = 150
)

tlm.render_plt(optics)
surface1.dump()

In [ ]:
part = tlm.lens_to_part(lens)
part